In [111]:
import gurobipy as gp
from gurobipy import GRB
import pandas as pd
from itertools import product

In [112]:
df_cases = pd.read_csv("cases.csv")
df_sessions = pd.read_csv("sessions.csv")

In [113]:
# Model data

# list of case IDs
cases = df_cases["CaseID"].tolist()
# list of OR session IDs
sessions = df_sessions["SessionID"].tolist()
# all possible combinations of cases and sessions
tasks = gp.tuplelist(list(product(cases, sessions)))
# expected duration for each operation
case_duration = pd.Series(df_cases["Expected Duration"].values, index=df_cases["CaseID"]).to_dict()
# duration of each OR room availability
session_duration = pd.Series(df_sessions["Duration"].values, index=df_sessions["SessionID"]).to_dict()
# session start times
df_sessions.loc[:, "Start"] = pd.to_timedelta(df_sessions["Start"])
df_sessions.loc[:, "Start"] = df_sessions["Start"].dt.total_seconds() / 60
session_start_time = pd.Series(df_sessions["Start"].values.round(2), index=df_sessions["SessionID"]).to_dict()
# case deadline
df_cases.loc[:, "TargetDeadline"] = pd.to_datetime(df_cases["TargetDeadline"], format="%d/%m/%Y")
df_cases.loc[:, "TargetDeadline"] = df_cases["TargetDeadline"].apply(lambda date: date.toordinal()) # Gregorian ordinal for the given DateTime object.
case_deadlines = pd.Series(df_cases["TargetDeadline"].values, index=df_cases["CaseID"]).to_dict()
# Session dates
df_sessions.loc[:, "Date"] = pd.to_datetime(df_sessions["Date"], format="%d/%m/%Y")
df_sessions.loc[:, "Date"] = df_sessions["Date"].apply(lambda date: date.toordinal()) # Gregorian ordinal for the given DateTime object.
session_dates = pd.Series(df_sessions["Date"].values, index=df_sessions["SessionID"]).to_dict()

disjunctions = []
for (case1, case2, session) in product(cases, cases, sessions):
    if (case1 != case2) and (case2, case1, session) not in disjunctions:
        disjunctions.append((case1, case2, session))

In [114]:
session_start_time

{1001: 510.0, 1002: 510.0, 1003: 510.0, 1004: 510.0}

In [115]:
#big M
M = 90000

# upper bound (minutes in a day)
upbound = 1440

# upper bound of session utilization set to 85%
max_util = 0.9 # .9 * total OR availability = 1782

# create model
model = gp.Model("OR Utilization")

# create decision variables
A = model.addVars(tasks, name="session_assigned", vtype=GRB.BINARY)

B = model.addVars(cases, name="case_start_time", lb=510, ub=upbound)

C = model.addVars(sessions, name="utilization", lb=0, ub=len(cases))

# set objective
model.setObjective(C.sum(), GRB.MAXIMIZE)

In [116]:
# constraint 1: start time of a case must be after the start time of the OR session it is assigned to
model.addConstrs((B[c] >= session_start_time[s] - (1 - A[(c,s)])*M for c,s in tasks), name="c1")

{(1, 1001): <gurobi.Constr *Awaiting Model Update*>,
 (1, 1002): <gurobi.Constr *Awaiting Model Update*>,
 (1, 1003): <gurobi.Constr *Awaiting Model Update*>,
 (1, 1004): <gurobi.Constr *Awaiting Model Update*>,
 (2, 1001): <gurobi.Constr *Awaiting Model Update*>,
 (2, 1002): <gurobi.Constr *Awaiting Model Update*>,
 (2, 1003): <gurobi.Constr *Awaiting Model Update*>,
 (2, 1004): <gurobi.Constr *Awaiting Model Update*>,
 (3, 1001): <gurobi.Constr *Awaiting Model Update*>,
 (3, 1002): <gurobi.Constr *Awaiting Model Update*>,
 (3, 1003): <gurobi.Constr *Awaiting Model Update*>,
 (3, 1004): <gurobi.Constr *Awaiting Model Update*>,
 (4, 1001): <gurobi.Constr *Awaiting Model Update*>,
 (4, 1002): <gurobi.Constr *Awaiting Model Update*>,
 (4, 1003): <gurobi.Constr *Awaiting Model Update*>,
 (4, 1004): <gurobi.Constr *Awaiting Model Update*>,
 (5, 1001): <gurobi.Constr *Awaiting Model Update*>,
 (5, 1002): <gurobi.Constr *Awaiting Model Update*>,
 (5, 1003): <gurobi.Constr *Awaiting Model Upd

In [117]:
# constraint 2: case end time must be before end time of OR session
model.addConstrs((B[c] + case_duration[c] <= session_start_time[s] + session_duration[s] + (1-A[(c,s)])*M for c,s in tasks), name="c2")

{(1, 1001): <gurobi.Constr *Awaiting Model Update*>,
 (1, 1002): <gurobi.Constr *Awaiting Model Update*>,
 (1, 1003): <gurobi.Constr *Awaiting Model Update*>,
 (1, 1004): <gurobi.Constr *Awaiting Model Update*>,
 (2, 1001): <gurobi.Constr *Awaiting Model Update*>,
 (2, 1002): <gurobi.Constr *Awaiting Model Update*>,
 (2, 1003): <gurobi.Constr *Awaiting Model Update*>,
 (2, 1004): <gurobi.Constr *Awaiting Model Update*>,
 (3, 1001): <gurobi.Constr *Awaiting Model Update*>,
 (3, 1002): <gurobi.Constr *Awaiting Model Update*>,
 (3, 1003): <gurobi.Constr *Awaiting Model Update*>,
 (3, 1004): <gurobi.Constr *Awaiting Model Update*>,
 (4, 1001): <gurobi.Constr *Awaiting Model Update*>,
 (4, 1002): <gurobi.Constr *Awaiting Model Update*>,
 (4, 1003): <gurobi.Constr *Awaiting Model Update*>,
 (4, 1004): <gurobi.Constr *Awaiting Model Update*>,
 (5, 1001): <gurobi.Constr *Awaiting Model Update*>,
 (5, 1002): <gurobi.Constr *Awaiting Model Update*>,
 (5, 1003): <gurobi.Constr *Awaiting Model Upd

In [118]:
# constraint 3: only one OR session per case (surgery entries need to be made scheduled only once)
for c in cases:
    model.addConstr((sum(A[(c,s)] for s in sessions) <= 1), "c3")

In [119]:
# constraint 4: surgeries have to be performed before their deadline
model.addConstrs((session_dates[s] <= case_deadlines[c] + (1-A[(c,s)])*M for c,s in tasks), name="c4")

{(1, 1001): <gurobi.Constr *Awaiting Model Update*>,
 (1, 1002): <gurobi.Constr *Awaiting Model Update*>,
 (1, 1003): <gurobi.Constr *Awaiting Model Update*>,
 (1, 1004): <gurobi.Constr *Awaiting Model Update*>,
 (2, 1001): <gurobi.Constr *Awaiting Model Update*>,
 (2, 1002): <gurobi.Constr *Awaiting Model Update*>,
 (2, 1003): <gurobi.Constr *Awaiting Model Update*>,
 (2, 1004): <gurobi.Constr *Awaiting Model Update*>,
 (3, 1001): <gurobi.Constr *Awaiting Model Update*>,
 (3, 1002): <gurobi.Constr *Awaiting Model Update*>,
 (3, 1003): <gurobi.Constr *Awaiting Model Update*>,
 (3, 1004): <gurobi.Constr *Awaiting Model Update*>,
 (4, 1001): <gurobi.Constr *Awaiting Model Update*>,
 (4, 1002): <gurobi.Constr *Awaiting Model Update*>,
 (4, 1003): <gurobi.Constr *Awaiting Model Update*>,
 (4, 1004): <gurobi.Constr *Awaiting Model Update*>,
 (5, 1001): <gurobi.Constr *Awaiting Model Update*>,
 (5, 1002): <gurobi.Constr *Awaiting Model Update*>,
 (5, 1003): <gurobi.Constr *Awaiting Model Upd

In [ ]:
# DO NOT RUN

#constraint 5: surgeries can't overlap for the same OR session

"""
Ex:

OR availability: {start}||||||||||------c1------||
                        ||||||||||              ||----5min + c2---||{end}
                        
CanNOT be:
OR availability: {start}||||||||||------c1------
                        ||||||||||           ----5min + c2---||{end}

                                        ^OVERLAP ABOVE^
"""

for c1, c2, s in disjunctions:
    model.addConstr(B + case_duration[c1] <= B + )

In [120]:
# constraint 6: linking utilization to # of cases assigned
model.addConstrs((C[s] == (1 / session_duration[s]) * sum(A[c,s] for c in cases) for s in sessions), name="c6")

{1001: <gurobi.Constr *Awaiting Model Update*>,
 1002: <gurobi.Constr *Awaiting Model Update*>,
 1003: <gurobi.Constr *Awaiting Model Update*>,
 1004: <gurobi.Constr *Awaiting Model Update*>}

In [121]:
model.optimize()

Gurobi Optimizer version 9.5.1 build v9.5.1rc2 (mac64[rosetta2])
Thread count: 10 physical cores, 10 logical processors, using up to 10 threads
Optimize a model with 394 rows, 154 columns and 844 nonzeros
Model fingerprint: 0x2e9df621
Variable types: 34 continuous, 120 integer (120 binary)
Coefficient statistics:
  Matrix range     [2e-03, 9e+04]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+03]
  RHS range        [1e+00, 9e+04]
Found heuristic solution: objective -0.0000000
Presolve removed 394 rows and 154 columns
Presolve time: 0.00s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.01 seconds (0.00 work units)
Thread count was 1 (of 10 available processors)

Solution count 2: 0.0974659 -0 
No other solutions better than 0.0974659

Optimal solution found (tolerance 1.00e-04)
Best objective 9.746588693957e-02, best bound 9.746588693957e-02, gap 0.0000%


In [122]:
#model.optimize()
for v in model.getVars():
    print(v.varName, v.x)

session_assigned[1,1001] 0.0
session_assigned[1,1002] 0.0
session_assigned[1,1003] 0.0
session_assigned[1,1004] 1.0
session_assigned[2,1001] 0.0
session_assigned[2,1002] 0.0
session_assigned[2,1003] 0.0
session_assigned[2,1004] 1.0
session_assigned[3,1001] 1.0
session_assigned[3,1002] 0.0
session_assigned[3,1003] 0.0
session_assigned[3,1004] 0.0
session_assigned[4,1001] 0.0
session_assigned[4,1002] 0.0
session_assigned[4,1003] 0.0
session_assigned[4,1004] 1.0
session_assigned[5,1001] 0.0
session_assigned[5,1002] 0.0
session_assigned[5,1003] 0.0
session_assigned[5,1004] 1.0
session_assigned[6,1001] 0.0
session_assigned[6,1002] 0.0
session_assigned[6,1003] 0.0
session_assigned[6,1004] 1.0
session_assigned[7,1001] 0.0
session_assigned[7,1002] 0.0
session_assigned[7,1003] 0.0
session_assigned[7,1004] 1.0
session_assigned[8,1001] 0.0
session_assigned[8,1002] 1.0
session_assigned[8,1003] 0.0
session_assigned[8,1004] 0.0
session_assigned[9,1001] 0.0
session_assigned[9,1002] 0.0
session_assign

In [76]:
print(model.objVal)

1782.0


In [29]:
session_start_time

{1001: 510.0, 1002: 510.0, 1003: 510.0, 1004: 510.0}

In [39]:
model.display()

Maximize
  <gurobi.LinExpr: utilization>
Subject To
  R0: <gurobi.LinExpr: -90000000.0 session_assigned + case_start_time> >= -8.99995e+07
  R1: <gurobi.LinExpr: -90000000.0 session_assigned + case_start_time> >= -8.99995e+07
  R2: <gurobi.LinExpr: -90000000.0 session_assigned + case_start_time> >= -8.99995e+07
  R3: <gurobi.LinExpr: -90000000.0 session_assigned + case_start_time> >= -8.99995e+07
  R4: <gurobi.LinExpr: -90000000.0 session_assigned + case_start_time> >= -8.99995e+07
  R5: <gurobi.LinExpr: -90000000.0 session_assigned + case_start_time> >= -8.99995e+07
  R6: <gurobi.LinExpr: -90000000.0 session_assigned + case_start_time> >= -8.99995e+07
  R7: <gurobi.LinExpr: -90000000.0 session_assigned + case_start_time> >= -8.99995e+07
  R8: <gurobi.LinExpr: -90000000.0 session_assigned + case_start_time> >= -8.99995e+07
  R9: <gurobi.LinExpr: -90000000.0 session_assigned + case_start_time> >= -8.99995e+07
  R10: <gurobi.LinExpr: -90000000.0 session_assigned + case_start_time> >= -8.

  R93: <gurobi.LinExpr: -90000000.0 session_assigned + case_start_time> >= -8.99995e+07
  R94: <gurobi.LinExpr: -90000000.0 session_assigned + case_start_time> >= -8.99995e+07
  R95: <gurobi.LinExpr: -90000000.0 session_assigned + case_start_time> >= -8.99995e+07
  R96: <gurobi.LinExpr: -90000000.0 session_assigned + case_start_time> >= -8.99995e+07
  R97: <gurobi.LinExpr: -90000000.0 session_assigned + case_start_time> >= -8.99995e+07
  R98: <gurobi.LinExpr: -90000000.0 session_assigned + case_start_time> >= -8.99995e+07
  R99: <gurobi.LinExpr: -90000000.0 session_assigned + case_start_time> >= -8.99995e+07
  R100: <gurobi.LinExpr: -90000000.0 session_assigned + case_start_time> >= -8.99995e+07
  R101: <gurobi.LinExpr: -90000000.0 session_assigned + case_start_time> >= -8.99995e+07
  R102: <gurobi.LinExpr: -90000000.0 session_assigned + case_start_time> >= -8.99995e+07
  R103: <gurobi.LinExpr: -90000000.0 session_assigned + case_start_time> >= -8.99995e+07
  R104: <gurobi.LinExpr: -90